# Simulação de memoria cache de mapeamento direto, completamente associativo, duas vias e quatro vias, por threads.


## Bibliotecas utilizadas


In [ ]:
import pandas as pd
from random import randint
import numpy as np

## Arquivo com as requisições feitas pela cpu


In [ ]:
#Mapeamento direto
cpu_requests = pd.read_csv("./enderecos.dat") # Lendo os dados de endereço de memoria

# Setando dados iniciais
erros = 0
acertos = 0
enderecos = len(cpu_requests['0'])
memoria = []

#
for i in range(enderecos):
  lista = [False, None]
  memoria.append(lista)

arq = cpu_requests['0']
addresses = []
for i in arq:
  addresses.append(int(i)) 

for line in arq:
  num = np.binary_repr(int(line), width=16)
  tag_bin = num[0:6]
  endereco_bin = num[6:14]
  word_bin = num[14:16]
  endereco = int(endereco_bin, 2)
  tag = int(tag_bin, 2)
  if memoria[endereco][0] == False:
    memoria[endereco][0] = True
    memoria[endereco][1] = tag
    erros += 1
  elif memoria[endereco][1] != tag:
    memoria[endereco][1] = tag
    erros += 1
  else:
    acertos += 1

In [ ]:
acuracy = acertos/enderecos
acuracy

0.8563125053423369

## Classes utilizadas 
- LinhaCacheMapeamentoDireto, nesta classe é onde é mapeado o número de endereços que podem ser armazenados por linha,  o método push insere um elemento em um posição que estava fazia nesta linha, em caso da linha esta cheia ele remove o primeiro que foi inserido. Os métodos sroll_to_right desloca os elementos da linha para a direita removendo assim o primeiro que foi inserido (FIFO).
- CacheMapeamentoDireto, é onde fica armazenada todas as linhas da mémoria cache. O metodo set_value_in_line insere um endereço na linha da cache.

- SimulacaoMapeamentoDireto, é onde é contabilizado o número de acertos de cache e de falhas de cache, um acerto é quando o endereço solicitado pelo cpu está presente na linha da cache, o caso de falha de cache é quanto o endereço não esta presente na linha da cache.

In [ ]:
class LinhaCacheMapeamentoDireto:
  def __init__(self,):
    self.flag = False
    self.columns = [None,None,None,None]
  
  def push(self,address):
    self.flag = True

    for index,value in enumerate(self.columns):
      if value == None:
        self.columns[index]=address 

      elif index == len(self.columns)-1:
        self.scroll_to_right(address)

  def scroll_to_right(self,address):
    for index in range(len(self.columns)-1):
      self.columns[index] = self.columns[index+1]
    self.columns[3] = address

  def __repr__(self):
    columns = [f"{index} : {value}" for index,value in enumerate(self.columns)]

    return f"{columns}"   
  

class CacheMapeamentoDireto:
  def __init__(self, number_lines=0):
    self.number_lines = number_lines
    self.memorie = [ LinhaCacheMapeamentoDireto() for index in range(self.number_lines)]
  
  def set_value_in_line(self,line,address):
      self.memorie[line].push(address)

  def __str__(self):
    rep = f"cache {self.number_lines} linhas\n"
    for index,line in enumerate(self.memorie):
      rep =rep+f" linha {index} e {line}\n"

    return rep


In [ ]:
cache = CacheMapeamentoDireto(number_lines = 1000)

def SimulacaoMapeamentoDireto(cache=cache, cpu_requests=cpu_requests):
  hit=0
  fault=0
  flag = True
  for request in cpu_requests["0"]:
    line = request%cache.number_lines
    if not cache.memorie[line].flag:
      fault=fault+1
    
    else:
      if request in cache.memorie[line].columns:
        hit = hit+1
        flag=False
      else:
        fault= fault+1
        flag = True

    if flag:
      cache.set_value_in_line(line,request)

  return hit
  #print(f"End simulate direct mapping number of successes {hit/enderecos} and number of fails {fault/enderecos}, number of requests {hit+fault}")

Resultado simulação em Mapeamento direto:

In [ ]:
SimulacaoMapeamentoDireto()

4976

## Cache de Mapeamento Associativo em conjunto de duas vias e quatro vias

In [ ]:
class LinhaCacheMapeamentoAssociativo2:
  def __init__(self,):
    self.way = [{"flag":False,"values":[None,None,None,None]},{"flag":False,"values":[None,None,None,None]}]
  
  def push(self,address):
    for key,value in enumerate(self.way):
      line = [block for block in value["values"]]
      
      if None in line:
        empty_index = line.index(None)
        self.way[key]["values"][empty_index] = address
        self.way[key]["flag"] = True
        
      elif key == 1: 
        rand_way = randint(0,1)
        rand_index = randint(0,3)
        self.way[rand_way]["values"][rand_index] = address
        self.way[rand_way]["flag"] = True

  def __repr__(self):
    columns = [f"way {index} : {value}" for index,value in enumerate(self.way)]

    return f"{columns}"   
  

class CacheMapeamentoAssociativo2:
  def __init__(self, number_lines=0):
    self.number_lines = number_lines
    self.memorie = [ LinhaCacheMapeamentoAssociativo2() for index in range(self.number_lines)]
  
  def set_value_in_line(self,line,address):
      self.memorie[line].push(address)

  def __str__(self):
    rep = f"cache {self.number_lines} linhas\n"
    for index,line in enumerate(self.memorie):
      rep =rep+f" linha {index} e {line}\n"

    return rep
class LinhaCacheMapeamentoAssociativo4:
  def __init__(self,):
    self.way = [{"flag":False,"values":[None,None,None,None]}, {"flag":False,"values":[None,None,None,None]}
                , {"flag":False,"values":[None,None,None,None]}, {"flag":False,"values":[None,None,None,None]}]
  
  def push(self,address):
    for key,value in enumerate(self.way):
      line = [block for block in value["values"]]
      
      if None in line:
        empty_index = line.index(None)
        self.way[key]["values"][empty_index] = address
        self.way[key]["flag"] = True
        
      elif key == 1: 
        rand_way = randint(0,1)
        rand_index = randint(0,3)
        self.way[rand_way]["values"][rand_index] = address
        self.way[rand_way]["flag"] = True

  def __repr__(self):
    columns = [f"way {index} : {value}" for index,value in enumerate(self.way)]

    return f"{columns}"   
  

class CacheMapeamentoAssociativo4:
  def __init__(self, number_lines=0):
    self.number_lines = number_lines
    self.memorie = [ LinhaCacheMapeamentoAssociativo4() for index in range(self.number_lines)]
  
  def set_value_in_line(self,line,address):
      self.memorie[line].push(address)

  def __str__(self):
    rep = f"cache {self.number_lines} linhas\n"
    for index,line in enumerate(self.memorie):
      rep =rep+f" linha {index} e {line}\n"

    return rep

In [ ]:
cache2_128 = CacheMapeamentoAssociativo2(number_lines = 128)
cache2_512 = CacheMapeamentoAssociativo2(number_lines = 512)
cache4_256 = CacheMapeamentoAssociativo4(number_lines = 256)
cache4_512 = CacheMapeamentoAssociativo4(number_lines = 512)

In [ ]:
def SimulacaoMapeamentoAssociativo2_128(cache=cache2_128, cpu_requests=cpu_requests):
  hit=0
  fault=0
  flag = True
  for request in cpu_requests["0"]:
    line = request%cache.number_lines

    flag_ways = set([block["flag"] for block  in cache.memorie[line].way])
    if False in flag_ways:
      fault = fault + 1

    else:
      if request in cache.memorie[line].way[0]["values"] or request in cache.memorie[line].way[1]["values"]:
        hit=hit+1
      else:
        fault = fault + 1
   
    if flag:
      cache.set_value_in_line(line,request)
  return hit
def SimulacaoMapeamentoAssociativo2_512(cache=cache2_512, cpu_requests=cpu_requests):
  hit=0
  fault=0
  flag = True
  for request in cpu_requests["0"]:
    line = request%cache.number_lines

    flag_ways = set([block["flag"] for block  in cache.memorie[line].way])
    if False in flag_ways:
      fault = fault + 1

    else:
      if request in cache.memorie[line].way[0]["values"] or request in cache.memorie[line].way[1]["values"]:
        hit=hit+1
      else:
        fault = fault + 1
   
    if flag:
      cache.set_value_in_line(line,request)
  return hit
  #print(f"End simulate associative mapping number of successes {hit} and number of fails {fault}, number of requests {hit+fault}")
def SimulacaoMapeamentoAssociativo4_256(cache=cache4_256, cpu_requests=cpu_requests):
    hit=0
    fault=0
    flag = True
    for request in cpu_requests["0"]:
      line = request%cache.number_lines

      flag_ways = set([block["flag"] for block  in cache.memorie[line].way])
      if False in flag_ways:
        fault = fault + 1

      else:
        if request in cache.memorie[line].way[0]["values"] or request in cache.memorie[line].way[1]["values"]:
          hit=hit+1
        else:
          fault = fault + 1
   
      if flag:
        cache.set_value_in_line(line,request)
    return hit
  #print(f"End simulate associative mapping number of successes {hit} and number of fails {fault}, number of requests {hit+fault}")
def SimulacaoMapeamentoAssociativo4_512(cache=cache4_512, cpu_requests=cpu_requests):
    hit=0
    fault=0
    flag = True
    for request in cpu_requests["0"]:
      line = request%cache.number_lines

      flag_ways = set([block["flag"] for block  in cache.memorie[line].way])
      if False in flag_ways:
        fault = fault + 1

      else:
        if request in cache.memorie[line].way[0]["values"] or request in cache.memorie[line].way[1]["values"]:
          hit=hit+1
        else:
          fault = fault + 1
   
      if flag:
        cache.set_value_in_line(line,request)
    return hit

Resultado simulação em Mapeamento associativo:

In [ ]:
print(SimulacaoMapeamentoAssociativo2_128())
print(SimulacaoMapeamentoAssociativo2_512())
print(SimulacaoMapeamentoAssociativo4_256())
print(SimulacaoMapeamentoAssociativo4_512())

4069
4853
4717
4875


In [ ]:
from collections import deque
def SimulacaoMapeamentoCompletamenteAssociativo():
    tags = [-1] * 4
    valid = [0] * 4
    LRU = deque()
    acertos = 0
    erros = 0
    total_instructions = 0

    for i in range(0, 2):
      for addr in addresses:
            offset = addr % 4
            tag = addr // (4)
            if tag in tags:
                location = tags.index(tag)
                if location in LRU:
                    LRU.remove(location)
                LRU.append(location)
                acertos += 1
            elif 0 in valid:
                location = valid.index(0)
                tags[location] = tag
                valid[location] = 1
                if i > 0:
                    erros += 1
                if location in LRU:
                    LRU.remove(location)
                LRU.append(location)
            else:
                leastUsedLoc = LRU.popleft()
                tags[leastUsedLoc] = tag
                if i > 0:
                    erros += 1
                if leastUsedLoc in LRU:
                    LRU.remove(leastUsedLoc)
                LRU.append(leastUsedLoc)
            if i > 0:
                total_instructions += 1
    return acertos*1/2

Resultado simulação em Mapeamento completamente associativo:

In [ ]:
SimulacaoMapeamentoCompletamenteAssociativo()

8774.0

In [ ]:
import threading
import random
import sys

In [ ]:
global x, x1, x2, x3, x4, x5

def task1():
    print("\nIniciando Thread {}".format(threading.current_thread().name))
    a = SimulacaoMapeamentoDireto()
    global x
    x = "\nAcertos para a Thread 1 (Direto): {} ".format(a) + " --> Porcentagem: " + str((a/enderecos)*100) + "%\nErros para a Thread 1: {} ".format(enderecos - a) + " --> Porcentagem: " + str(((enderecos - a)/enderecos)*100) + "%\n"
  
def task2():
    print("\nIniciando Thread {}".format(threading.current_thread().name))
    a = SimulacaoMapeamentoCompletamenteAssociativo()
    global x1
    x1 = "\nAcertos para a Thread 2 (Associativo): {} ".format(a) + " --> Porcentagem: " + str((a/enderecos)*100) + "%\nErros para a Thread 2: {} ".format(enderecos - a) + " --> Porcentagem: " + str(((enderecos - a)/enderecos)*100) + "%\n"
    
def task3():
    print("\nIniciando Thread {}".format(threading.current_thread().name))
    a = SimulacaoMapeamentoAssociativo2_512()
    global x2
    x2 = "\nAcertos para a Thread 3 (2 vias - 512): {} ".format(a) + " --> Porcentagem: " + str((a/enderecos)*100) + "%\nErros para a Thread 3: {} ".format(enderecos - a) + " --> Porcentagem: " + str(((enderecos - a)/enderecos)*100) + "%\n"
    
def task4():
    print("\nIniciando Thread {}".format(threading.current_thread().name))
    a = SimulacaoMapeamentoAssociativo4_256()
    global x3
    x3 = "\nAcertos para a Thread 4 (4 vias - 256): {} ".format(a) + " --> Porcentagem: " + str((a/enderecos)*100) + "%\nErros para a Thread 4: {} ".format(enderecos - a) + " --> Porcentagem: " + str(((enderecos - a)/enderecos)*100) + "%\n"
    
def task5():
    print("\nIniciando Thread {}".format(threading.current_thread().name))
    a = SimulacaoMapeamentoAssociativo2_128()
    global x4
    x4 = "\nAcertos para a Thread 5 (2 vias - 128): {} ".format(a) + " --> Porcentagem: " + str((a/enderecos)*100) + "%\nErros para a Thread 5: {} ".format(enderecos - a) + " --> Porcentagem: " + str(((enderecos - a)/enderecos)*100) + "%\n"
   
def task6():
    print("\nIniciando Thread {}".format(threading.current_thread().name))
    a = SimulacaoMapeamentoAssociativo4_512()
    global x5
    x5 = "\nAcertos para a Thread 6 (4 vias - 512): {} ".format(a) + " --> Porcentagem: " + str((a/enderecos)*100) + "%\nErros para a Thread 6: {} ".format(enderecos - a) + " --> Porcentagem: " + str(((enderecos - a)/enderecos)*100) + "%\n"
    

def main_task():
    global x, x1, x2, x3, x4, x5
  # print name of main thread
    print("Main thread name: {}".format(threading.main_thread().name))
  # creating threads
    t1 = threading.Thread(target=task1, name='1')
    t2 = threading.Thread(target=task2, name='2') 
    t3 = threading.Thread(target=task3, name='3') 
    t4 = threading.Thread(target=task4, name='4') 
    # extras (tests with 2 ways (128) and 4 ways (512))
    t5 = threading.Thread(target=task5, name='5') 
    t6 = threading.Thread(target=task6, name='6') 
    # starting threads
    t1.start()
    t2.start()
    t3.start()
    t4.start()
    t5.start()
    t6.start()
    # wait until all threads finish
    t1.join()
    t2.join()
    t3.join()
    t4.join()
    t5.join()
    t6.join()
    # Results
    print(x + x1 + x2 + x3 + "\n##########  Extras  ##########" + x4 + x5)

if __name__ == "__main__": 
    main_task()


Main thread name: MainThread

Iniciando Thread 1

Iniciando Thread 2

Iniciando Thread 3

Iniciando Thread 4

Iniciando Thread 5

Iniciando Thread 6

Acertos para a Thread 1 (Direto): 4976  --> Porcentagem: 42.53354987605778%
Erros para a Thread 1: 6723  --> Porcentagem: 57.46645012394222%

Acertos para a Thread 2 (Associativo): 8774.0  --> Porcentagem: 74.99786306521925%
Erros para a Thread 2: 2925.0  --> Porcentagem: 25.00213693478075%

Acertos para a Thread 3 (2 vias - 512): 5627  --> Porcentagem: 48.09812804513206%
Erros para a Thread 3: 6072  --> Porcentagem: 51.90187195486794%

Acertos para a Thread 4 (4 vias - 256): 4702  --> Porcentagem: 40.19146935635524%
Erros para a Thread 4: 6997  --> Porcentagem: 59.80853064364475%

##########  Extras  ##########
Acertos para a Thread 5 (2 vias - 128): 4033  --> Porcentagem: 34.47303188306693%
Erros para a Thread 5: 7666  --> Porcentagem: 65.52696811693308%

Acertos para a Thread 6 (4 vias - 512): 5665  --> Porcentagem: 48.42294213180614%
